In [1]:
#dash modules
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

#plotly modules
import plotly.offline as pyo 
import plotly.figure_factory as ff
import plotly.graph_objs as go 
from plotly import tools

#dataset modules
import pandas as pd
import numpy as np

## 1. Single Callbacks for Interactivity
How to interact and connect to those components through the use of callbacks

In [ ]:
app = dash.Dash()

app.layout = html.Div([
    dcc.Input(id='my-id',value='Initial Text',type='text'), html.Div(id='my-div')
])

@app.callback(Output(component_id='my-div', component_property='children'), [Input(component_id = 'my-id', component_property='value')])
def update_output_div(input_value):
    return "You entered: {}".format(input_value)

if __name__ == '__main__':
    app.run_server()

## 2. Dash Callbacks for Graphs
More complex code in which you can interact with the plot, filtering it, for example.

In [2]:
df = pd.read_csv('Data/gapminderDataFiveYear.csv')
df.head()

,country,year,pop,continent,lifeExp,gdpPercap
0,Afghanistan,1952,8425333.0,Asia,28.801,779.445314
1,Afghanistan,1957,9240934.0,Asia,30.332,820.853030
2,Afghanistan,1962,10267083.0,Asia,31.997,853.100710
3,Afghanistan,1967,11537966.0,Asia,34.020,836.197138
4,Afghanistan,1972,13079460.0,Asia,36.088,739.981106


In [3]:
app = dash.Dash()

year_options = [{'label': str(year), 'value': year} for year in df['year'].unique()]

app.layout = html.Div([
    dcc.Graph(id='graph'),
    dcc.Dropdown(id='year-picker', options=year_options, value =df['year'].min())
])

@app.callback(Output(component_id='graph',component_property='figure'),[Input('year-picker','value')])

def update_figure(selected_year):

    #Data only for selected year from dropdown
    filtered_df = df[df['year']==selected_year]

    traces = []

    for continent_name in filtered_df['continent'].unique():
        df_by_continent = filtered_df[filtered_df['continent']==continent_name]
        traces.append(
            go.Scatter(
                x = df_by_continent['gdpPercap'],
                y = df_by_continent['lifeExp'],
                mode ='markers',
                opacity=0.7,
                marker = {'size':15},
                name = continent_name
            )
        )
    return {'data': traces, 'layout': go.Layout(title='My plot', xaxis ={'title': 'GDP Per Cap', 'type': 'log'}, yaxis = {'title': 'Life Expectancy'})}

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


In [ ]:
'''def decorator(func):
   return func

@decorator
def some_func():
    pass

Is equivalent to this code:

def decorator(func):
    return func

def some_func():
    pass

some_func = decorator(some_func)'''

## 3. Multiple Inputs
The input goes inside of a list because you can connect multiple input to one single app.callback output in a graph

In [9]:
df = pd.read_csv('Data/mpg.csv')

app = dash.Dash()

#['mpg','hp','displace'...]
features = df.columns

# we will have three main elements in the dash: 1- the first dropdown, 2- the second dropdwon, 3-the graph
# it is better to have a div to each dropdown, so you can style each one individually
#without a style one dropdown will appear on top of the other, because by default it takes the entire width of the web page
#the style is associate with the div, so it's outside of the brakets
app.layout = html.Div([
    html.Div([
        dcc.Dropdown(id='xaxis',
                    options=[{'labels': str(col), 'value': col} for col in features],
                    value='displacement')
    ],style={'width':'48%', 'display':'inline-block'}),
    html.Div([
        dcc.Dropdown(id='yaxis',
                    options=[{'labels': str(col), 'value': col} for col in features],
                    value='mpg')
    ],style={'width':'48%', 'display':'inline-block'}),
    dcc.Graph(id='feature-graphic')
],style={'padding':10})

#app.layout()

@app.callback(Output('feature-graphic','figure'),[Input('xaxis','value'),Input('yaxis','value')])
def update_graph(xaxis_name,yaxis_name):
    trace = [go.Scatter(x=df[xaxis_name],
                        y=df[yaxis_name],
                        text=df['name'],
                        mode='markers',
                        marker={'size':15})]
    layout = go.Layout(title='My Dashboard for MPG', xaxis ={'title': xaxis_name, 'type': 'log'}, yaxis = {'title': yaxis_name})
    return {'data':trace,'layout':layout}

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Feb/2021 21:02:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2021 21:02:23] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2021 21:02:23] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Feb/2021 21:02:23] "POST /_dash-update-component HTTP/1.1" 200 -


## 4. Multiple Outputs
The idea is that you can have different outputs to the same input. So, in this example, you have the text output and the image output for the same input.

In [18]:
df = pd.read_csv('Data/wheels.csv')

app = dash.Dash()

def encode_image(image_file):
    encoded = base64.b6b64encode(open(image_file,'rb').read())
    return 'data:image/png;base64,{}'.format(encoded.decode())

app.layout = html.Div([
    dcc.RadioItems(id='wheels',
                   options=[{'label': str(wheel), 'value': wheel} for wheel in df['wheels'].unique()],
                   value=1),
    html.Div(id='wheels-ouput'),
    html.Hr(),
    dcc.RadioItems(id='colors',
    options=[{'label': str(color), 'value': color} for color in df['color'].unique()],
                   value='blue'),
    html.Div(id='colors-output'),
    html.Img(id='display-image',src='children',height=300)
],style={'fontfamily':'helvetica','fontsize':18})

@app.callback(Output('wheels-ouput','children'),
            [Input('wheels','value')])
def callback_wheels(wheels_value):
    return "you chose {}".format(wheels_value)

@app.callback(Output('colors-output','children'),
            [Input('colors','value')])
def callback_colors(colors_value):
    return "you chose {}".format(colors_value)

@app.callback(Output('display-image','src'),
            [Input('wheels','value'),
            Input('colors','value')])
def callback_image(wheel,color):
    path = 'Data/iamges'
    return encode_image(path+df[(df['wheels']==wheel) &(df['color']==color)]['image'].values[0])

if __name__ == '__main__':
    app.run_server()

defined
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\CarolinaBaptistadeLi\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\CarolinaBaptistadeLi\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\CarolinaBaptistadeLi\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\CarolinaBaptistadeLi\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\CarolinaBaptistadeLi\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\CarolinaBaptistadeLi\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C

### Exercise: Interactive Components
- Objective: Create a dashboard that takes in two or more input values and returns their product as the output.

In [ ]:
'''
import dash
import dash_html_components as html
import dash_core_components as dcc

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    dcc.RangeSlider(
        id='my-range-slider',
        min=0,
        max=20,
        step=0.5,
        value=[5, 15]
    ),
    html.Div(id='output-container-range-slider')
])


@app.callback(
    dash.dependencies.Output('output-container-range-slider', 'children'),
    [dash.dependencies.Input('my-range-slider', 'value')])
def update_output(value):
    return 'You have selected "{}"'.format(value)


if __name__ == '__main__':
    app.run_server(debug=True)
'''

In [24]:
numbers_list = [*range(-20,20,1)]
print(numbers_list)

[-20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [31]:
app = dash.Dash()

#numbers_list = [*range(-20,20,1)]

app.layout = html.Div([
    dcc.RangeSlider(
        id='my-range-slider',
        min=-20,
        max=20,
        step=1,
        value=[5, 15],
        marks = {i: '{}'.format(i) for i in range(-20,20,1)}
    ),
    html.Div(id='output-container-range-slider')
])

@app.callback(
    dash.dependencies.Output('output-container-range-slider', 'children'),
    [dash.dependencies.Input('my-range-slider', 'value')])
def update_output(value):
    result = value[0]*value[1]
    return 'You have selected "{}"'.format(result)

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Envi